In [20]:
import strax


import XeSim

from plugins.micro_physics.input import ChunkInput
from XeSim.plugins.micro_physics.find_cluster import FindCluster
from XeSim.plugins.micro_physics.merge_cluster import MergeCluster
from XeSim.plugins.micro_physics.electric_field import ElectricField
from XeSim.plugins.micro_physics.yields import NestYields, BBFYields
from XeSim.plugins.micro_physics.microphysics_summary import MicroPhysicsSummary

ModuleNotFoundError: No module named 'XeSim'

In [2]:
import XeSim

ModuleNotFoundError: No module named 'straxen'

In [2]:
XeSim

<module 'XeSim' from '/Users/henningse/miniforge3/lib/python3.9/site-packages/XeSim/__init__.py'>

In [ ]:
st = strax.Context(register = [ChunkInput,
                               FindCluster,
                               MergeCluster,
                               ElectricField,
                               NestYields,
                               #BBFYields,
                               MicroPhysicsSummary],
                   storage = [strax.DataDirectory('/scratch/midway2/hschulze/epix_data/')]
                  )

st.set_config({"path": "/project2/lgrandi/xenonnt/simulations/testing",
               "file_name": "pmt_neutrons_100.root",
               "ChunkSize": 50
              })


st.make("00000","geant4_interactions")
st.make("00000","cluster_index")
st.make("00000","clustered_interactions")
st.make("00000",[ "electric_field_values"])
st.make("00000",[ "quanta"])
st.make("00000",[ "microphysics_summary"])

In [ ]:
geant4_interactions = st.get_df("00000",["geant4_interactions", "cluster_index"])
geant4_interactions

In [ ]:
clustered_interactions = st.get_df("00000",["clustered_interactions", "electric_field_values", "quanta"])
clustered_interactions

In [ ]:
microphysics_summary = st.get_df("00000",[ "microphysics_summary"])
microphysics_summary

## S1 and S2

In [ ]:
import cutax

In [ ]:
from XeSim.plugins.detector_physics.S1_Signal import S1PhotonPropagation
from XeSim.plugins.detector_physics.electron_drift import ElectronDrift
from XeSim.plugins.detector_physics.electron_extraction import ElectronExtraction
from XeSim.plugins.detector_physics.electron_timing import ElectronTiming
from XeSim.plugins.detector_physics.secondary_scintillation import SecondaryScintillation
from XeSim.plugins.detector_physics.S2_Signal import S2PhotonPropagation

from XeSim.plugins.pmt_and_daq.pmt_afterpulses import PMTAfterPulses

from XeSim.plugins.pmt_and_daq.pmt_response_and_daq import PMTResponseAndDAQ

In [ ]:
st = cutax.contexts.xenonnt_sim_SR0v3_cmt_v9(output_folder = "/scratch/midway2/hschulze/RefactorOutput")
st.register(ChunkRootFile)
st.register(FindCluster)
st.register(MergeCluster)
st.register(ElectricField)
st.register(NestYields)
st.register(MicroPhysicsSummary)

st.register(S1PhotonPropagation)

st.register(ElectronDrift)
st.register(ElectronExtraction)
st.register(ElectronTiming)
st.register(SecondaryScintillation)
st.register(S2PhotonPropagation)

st.register(PMTAfterPulses)

st.register(PMTResponseAndDAQ)


st.set_config({"path": "/project2/lgrandi/xenonnt/simulations/testing",
               "file_name": "pmt_neutrons_100.root",
               "ChunkSize": 20
              })

In [ ]:
st.make("00000","geant4_interactions")
st.make("00000","cluster_index")
st.make("00000","clustered_interactions")
st.make("00000",[ "electric_field_values"])
st.make("00000",[ "quanta"])
st.make("00000",[ "microphysics_summary"])

In [ ]:
#S1
st.make("00000",[ "S1_channel_and_timings"])

#S2
st.make("00000","drifted_electrons")
st.make("00000","extracted_electrons")
st.make("00000","electron_time")
st.make("00000","photons")
st.make("00000","sum_photons")
st.make("00000","photon_channels_and_timeing")

#Afterpulses
st.make("00000","pmt_afterpulses")

In [ ]:
#RawRecords
st.make("00000","raw_records")

In [ ]:
# From here you should be able to get any straxen data you want
st.make("00000","event_info")